In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display


from sklearn.experimental import enable_iterative_imputer  # импорт IterativeImputer

# Процессинг
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# CT
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV

# Moдельки
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Meтрики
from sklearn.metrics import mean_absolute_percentage_error, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, make_scorer

# стырил из пар 
import category_encoders as ce

# убрать варнинги
import warnings
warnings.filterwarnings('ignore')



### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)

# Предобработка данных 
# 4 значения
df = df[(df['table'] >= 40) & (df['table'] <= 80)]
df = df[(df['depth'] >= 45) & (df['depth'] <= 75)]
# 14 Нул значений
df = df.drop(df[df["x"]==0].index)
df = df.drop(df[df["y"]==0].index)
df = df.drop(df[df["z"]==0].index)
# 78 дубликатов
df = df.drop_duplicates()

df = df[(df['x'] >= 2) & (df['x'] <= 11)]
df = df[(df['y'] >= 2) & (df['y'] <= 11)]
df = df[(df['z'] >= 2) & (df['z'] <= 7)]

# Доработка данных
X = df.drop(['price'], axis=1)
y = df['price']                 

# Всемогущий препроцессинг 

# Порядок признаков важен 
num_features = ['carat', 'depth', 'table']
axis_features = ['x', 'y', 'z']
clarity_map = [['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']]
color_map = [['J', 'I', 'H', 'G', 'F', 'E', 'D']]
cut_map = [['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']]


# Числовые признаки
num_pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())  # Применение MinMaxScaler, ткв GridSearchCV
])

clarity_pipeline = Pipeline(steps=[
    ('ce', OrdinalEncoder(categories=clarity_map)),  
    ('scaler', PowerTransformer())
])


color_pipeline = Pipeline(steps=[
    ('ce', OrdinalEncoder(categories=color_map)),  
    ('scaler', StandardScaler())
])

cut_pipeline = Pipeline(steps=[
    ('ce', OrdinalEncoder(categories=cut_map)),  
    ('scaler', MinMaxScaler())  
])

axis_pipeline = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=20)),  
    ('poly', PolynomialFeatures(degree=2, include_bias=False)), 
    ('scaler', StandardScaler())  
])

CT = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, num_features),
        ("cut", cut_pipeline, ['cut']),
        ("axis", axis_pipeline, axis_features),
        ('color', color_pipeline, ['color']),
        ('clarity', clarity_pipeline, ['clarity'])
    ],
    remainder='drop'
).set_output(transform='pandas')


In [6]:
# Блок обучения модели

In [ ]:
model = TransformedTargetRegressor(
    regressor=Pipeline(steps=[
        ('preprocessing', CT),  
        ('estimator', KNeighborsRegressor(n_neighbors=11, p=5, weights='distance')  
    ]),
    transformer=StandardScaler()
)

model.fit(X, y)

In [7]:
# Блок предсказания с использованием тестового набора

In [ ]:
# Наши любимые метрики (очень надеюсь, что не налажал тут) 

# X_test = pd.read_csv(path_test)
# X = df.drop(['price'], axis=1)
# y = df['price']   
# y_test у Вас 

In [ ]:
y_pred = model.predict(X) 

In [ ]:
def irq_mse(y_test, y_pred):
    delta = y_pred - y_test
    Q25 = np.quantile(delta, 0.25)
    Q75 = np.quantile(delta, 0.75)
    irq = Q75 - Q25
    mask = (delta < (Q25 - 1.5 * irq)) | (delta > (Q75 + 1.5 * irq))
    if (mask.sum() == 0):
        rez = 0
    else:
        rez = -sum((delta[mask]) ** 2) / mask.sum()
    return rez

In [ ]:
print(f"IQR MSE: {irq_mse(y_test, y_pred)}")

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict 